In [27]:
api_key = 

In [3]:
import requests


In [47]:
## récupère puis stocke le PUUID du summoner

def ranking (summoner_list):

    for summoner in summoner_list:
        data_summoner = requests.get('https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+ summoner +'?api_key='+api_key).json()
        if data_summoner is not None and "id" in data_summoner:
            id_summoner = data_summoner.get("id")
            puuid_summoner = data_summoner.get("puuid")
            #print(puuid_summoner,id_summoner)
  
            ## récupère les matchs (les derniers) du summoner (ici 5 matchs)
            ## ressource utile les ID des queues : https://static.developer.riotgames.com/docs/lol/queues.json
            ## récupère la division, le tier et les LP du compte
            
            data_rank = requests.get('https://euw1.api.riotgames.com/lol/league/v4/entries/by-summoner/'+ id_summoner +'?api_key='+api_key).json()
            print(data_rank)

            for data_SOLOQ in data_rank:
                #if 'RANKED_SOLO_5x5' in data_SOLOQ['queueType']:
                if data_SOLOQ['queueType'] == "RANKED_SOLO_5x5":
                    data_rank=data_SOLOQ
                    print(summoner,data_rank['tier'],data_rank['rank'],data_rank['leaguePoints'], "LP")
                if 'RANKED_SOLO_5x5' not in data_rank:
                    print("pas de SOLO Q")
        else:
            print(f"Failed to retrieve data for summoner {summoner}")

#summoner_list = ["SaintLoutr","Draeknof","Haukeas Grémory","Caio73","charlos92604","7689000","Alrsay","TheK1ngPotato","Blarckful"]
summoner_list = ["SaintLoutr","TheK1ngPotato","Blarckful"]
ranking(summoner_list)
    




[{'leagueId': 'ffb5c0c3-5a9e-4422-b342-1d4709120164', 'queueType': 'RANKED_FLEX_SR', 'tier': 'SILVER', 'rank': 'I', 'summonerId': 'ehqKNeSs9tBC41Zni3yOrKcsPH6daMFI1RJXN-ovYffT4SMnCJwcTbLF_g', 'summonerName': 'SaintLoutr', 'leaguePoints': 6, 'wins': 28, 'losses': 13, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}, {'leagueId': '0c6e0dbf-7db9-4fe9-ad2e-d03e751bac52', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'SILVER', 'rank': 'III', 'summonerId': 'ehqKNeSs9tBC41Zni3yOrKcsPH6daMFI1RJXN-ovYffT4SMnCJwcTbLF_g', 'summonerName': 'SaintLoutr', 'leaguePoints': 54, 'wins': 56, 'losses': 56, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}]
pas de SOLO Q
SaintLoutr SILVER III 54 LP
pas de SOLO Q
[]
[{'leagueId': '475dbc4d-10cb-4ac2-9ce7-aa680335c6f0', 'queueType': 'RANKED_FLEX_SR', 'tier': 'SILVER', 'rank': 'III', 'summonerId': 'frBmwKC8AWryZwu6st0zS8Fcm7swCNFHgdNXFDmdaCMJkdE', 'summonerName': 'Blarckful', 'leaguePoints': 3, 'wins': 26, 'lo

In [5]:
##vérifie le résultat des game avec l'ID du match et le PUUID du summoner 
def result_match (summoner_list):
    
    for summoner in summoner_list:
        data_summoner = requests.get('https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+ summoner +'?api_key='+api_key).json()
        puuid_summoner = data_summoner.get("puuid")
        matches_id = requests.get('https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/'+ puuid_summoner +'/ids?queue='+'420'+'&start=0&count=1&api_key='+api_key).json()
        print(summoner)

        
        for match_id in matches_id:
            match_info = requests.get("https://europe.api.riotgames.com/lol/match/v5/matches/"+ match_id +"?api_key=" + api_key).json()
            for player in (match_info['info']['participants']):
                if player['puuid'] == puuid_summoner:
                    if player['win'] == True:
                        print("WIN")
                    else:
                        print("LOOSE")
                
    list_player = []

summoner_list = ["SaintLoutr","Draeknof","Haukeas Grémory"]
result_match(summoner_list)

SaintLoutr
LOOSE
WIN
LOOSE
LOOSE
LOOSE
Draeknof
LOOSE
WIN
WIN
LOOSE
WIN
Haukeas Grémory
LOOSE
LOOSE
WIN
WIN
WIN


IndentationError: unexpected indent (1143700073.py, line 14)